This example uses the default GLiNER model to perform named entity recognition.



In [2]:
import json
import numpy as np
from pathlib import Path
from pydantic import BaseModel, field_validator
from typing import List
from sklearn.metrics import (
    jaccard_score,
    hamming_loss,
    f1_score,
    precision_score,
    recall_score,
    accuracy_score,
)
from gliner import GLiNER

c:\Users\vonbr\Documents\playground\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class Entity(BaseModel):
    entity: str
    types: str
    
    @field_validator('entity')
    @classmethod
    def clean_entity(cls, v):
        # Take only the text before the comma if there is one
        if isinstance(v, str) and "," in v:
            return v.split(",")[0].strip()
        return v

class Result(BaseModel):
    text: str
    entities: List[Entity]

labels = ["Interval", "Organization", "Money", "Date", "Phone", "Address", "Person", "Faculty", "PaymentMethod", "Email", "Gift Type", "Frequency", "Distribution"]

In [4]:
test_set_path = Path("results.jsonl")

results = []

with open(test_set_path, "r") as f:
    for line in f:
        raw_data = json.loads(line)
        try:
            result = Result(**raw_data)
            results.append(result)

            filtered_entities = [entity for entity in result.entities if entity.types in labels]
            
            result.entities = filtered_entities
            results.append(result)
        except Exception as e:
            print(f"Error parsing line: {e}")
            print(f"Problematic data: {raw_data}")

In [5]:
model = GLiNER.from_pretrained("urchade/gliner_medium-v2.1")

ner_results = []
for result in results:
    text = result.text
    entities = model.predict_entities(text, labels)
    classified = [Entity(entity=entity["text"], types=entity["label"]) for entity in entities]
    ner_results.append(Result(text=text, entities=classified))

Fetching 5 files: 100%|██████████| 5/5 [00:00<?, ?it/s]
c:\Users\vonbr\Documents\playground\.venv\Lib\site-packages\transformers\convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [9]:
labels = sorted(labels)

def sort_entities_by_type(results):
    for result in results:
        result.entities.sort(key=lambda x: x.types)
    return results

def multi_hot_encode(entities, labels):
    vector = np.zeros(len(labels), dtype=int)
    for entity in entities:
        if entity.types in labels:
            vector[labels.index(entity.types)] = 1
    return vector

def hamming_score(y_true, y_pred):
    return 1 - hamming_loss(y_true, y_pred)

results = sort_entities_by_type(results)
ner_results = sort_entities_by_type(ner_results)

y_true = np.array([multi_hot_encode(gt.entities, labels) for gt in results])
y_pred = np.array([multi_hot_encode(pred.entities, labels) for pred in ner_results])

jaccard_macro = jaccard_score(y_true, y_pred, average='macro')
jaccard_micro = jaccard_score(y_true, y_pred, average='micro')
jaccard_samples = jaccard_score(y_true, y_pred, average='samples')
hamming_macro = hamming_score(y_true, y_pred)

f1_macro = f1_score(y_true, y_pred, average="macro")
f1_micro = f1_score(y_true, y_pred, average="micro")
f1_samples = f1_score(y_true, y_pred, average="samples")

precision_macro = precision_score(y_true, y_pred, average="macro", zero_division=0)
precision_micro = precision_score(y_true, y_pred, average="micro", zero_division=0)
precision_samples = precision_score(y_true, y_pred, average="samples", zero_division=0)

recall_macro = recall_score(y_true, y_pred, average="macro", zero_division=0)
recall_micro = recall_score(y_true, y_pred, average="micro", zero_division=0)
recall_samples = recall_score(y_true, y_pred, average="samples", zero_division=0)

accuracy = accuracy_score(y_true, y_pred)


# Print results
print(f"Jaccard Score (Macro): {jaccard_macro:.4f}")
print(f"Jaccard Score (Micro): {jaccard_micro:.4f}")
print(f"Jaccard Score (Samples): {jaccard_samples:.4f}")
print(f"Hamming Score (Macro): {hamming_macro:.4f}")
print(f"F1 Score (Macro): {f1_macro:.4f}")
print(f"F1 Score (Micro): {f1_micro:.4f}")
print(f"F1 Score (Samples): {f1_samples:.4f}")
print(f"Precision (Macro): {precision_macro:.4f}")
print(f"Precision (Micro): {precision_micro:.4f}")
print(f"Precision (Samples): {precision_samples:.4f}")
print(f"Recall (Macro): {recall_macro:.4f}")
print(f"Recall (Micro): {recall_micro:.4f}")
print(f"Recall (Samples): {recall_samples:.4f}")
print(f"Accuracy: {accuracy:.4f}")

Jaccard Score (Macro): 0.5689
Jaccard Score (Micro): 0.6064
Jaccard Score (Samples): 0.6275
Hamming Score (Macro): 0.8192
F1 Score (Macro): 0.6418
F1 Score (Micro): 0.7550
F1 Score (Samples): 0.7515
Precision (Macro): 0.7953
Precision (Micro): 0.7621
Precision (Samples): 0.7846
Recall (Macro): 0.6757
Recall (Micro): 0.7479
Recall (Samples): 0.7497
Accuracy: 0.0800
